In [166]:
# Libraries
import json
import jsonschema
import os
import numpy as np
import itertools

## Create JSON data for the evacuation

### Color coding:
* <span style="color:lightblue">▉</span> Good but don't rush or consider
* <span style="color:lightgreen">▉</span> Not high prio but might be useful for debugging
* <span style="color:yellow">▉</span> Asap, where the testing will actually occur

### Params to test

#### <span style="color:lightblue">▉</span> Parameters
* Sim type : For extension
#### <span style="color:lightgreen">▉</span> Parameters
* Dims : Maybe just for different scenarios
* Map_seed if needed to make a map fixed (Don't really consider)
* evac_number
#### <span style="color:yellow">▉</span> Parameters
* Dynamic movement influence
* Aspiration influence and equation
* Ratio influence and equation
* fire spread
* lc/ld
* static influence

### Create validator function

In [167]:
schema = {
    "dim" : {
        "type" : "array",
        "maxItems" : 2,
        "items" : {
            "type" : "integer"
        }
    },
    "sim_type" : {
        "type" : "string",
        "items" : { "enum" : ["Flow" , "Total"]}
    },
    "param_seed" : {
        "type" : ["integer", "null"]
    },
    "setup" : {
        "type" : "object",
        "properties" : {
            "map_seed" : { "type" : ["number", "null"] },
            "initial_fire" : {
                "type" : ["array" , "null"],
                "maxItems" : 2,
                "items" : {
                    "type" : "integer"
                }
            },
            "initial_evac" : {
                "type" : ["array" , "null"],
                "items" : {
                    "type" : {
                        "type" : "object",
                        "properties" : {
                            "strategy" : {
                                "type" : "string",
                                "items" : {"enum" : ["Competititve" , "Cooperative"]}
                            },
                            "x" : {
                                "type" : "integer"
                            },
                            "y" : {
                                "type" : "integer"
                            },
                            "pr_c" : {
                                "type"  : "float"
                            }
                        }                        
                    }
                }
            },
            "evac_number" : {
                "type" : "integer"
            },
            "fire_spread" : {
                "type" : ["number","null"]
            },
            "lc" : {
                "type" : ["number","null"]
            },
            "ld" : {
                "type" : ["number","null"]
            },
        }
    },
    "fire" : {
        "type" : "object",
        "properties" : {
            "frontier" : {
                "type" : ["string" , "null"],
                "items" : {
                    "enum" : ["VecTree"]
                }
            },
            "movement" : {
                "type" : [
                    {
                        "type" : "object",
                        "properties" : {
                            "ClosestDistance" : {
                                "type" : ["number", "null"]
                            }
                        }
                    },
                ]
            },
            "aspiration" : {
                "type" : [
                    {
                        "type" : "object",
                        "properties" : {
                            "LogAspiration" :   {"type" : ["number", "null"]} 
                        }
                    }
                ]
            },
            "ratio" : {
                "type" : "object",
                "properties" : {
                    "Root" : {
            "type" : ["number",  "null"]
                    }
                }
            }
        }
    },
    "escape" : {
        "type" : "string",
        "items" : {"enum" : ["TimeTracker"]}
    },
    "death" : {
        "type" : "string",
        "items" : {"enum" : ["AnnounceInput"]}
    }, 
    "static_input" : {
        "type" : [
            {
                "type" : "object",
                "properties" : {
                    "ClosestToExit" : { "type" : ["number", "null"]}
                }
            }
        ]
    },
}

validator = jsonschema.Draft202012Validator(schema)
path = "../../inputs/tests/"
filename = "base_input.json"
with open(f"{path}{filename}") as f:
    js_data = json.load(f)
    print(validator.is_valid(js_data))
names = []
filedir = ""

True


In [168]:
# Default values
sim_type = "Total"
n = 21
dims = [n,n]
escape = "TimeTracker"
death = "AnnounceInput"
param_seed = 10

In [169]:
# Populations density
perc = [0.1]
evac_numbers = []
for p in perc:
    evac_numbers.append(int(n * n * p))

In [170]:
# Learning rates
# lc_over_ld = [0.5]
# lds = [0.4]
# lcldvals = []
# for rat in lc_over_ld:
#     for ld in lds:
#         lcldvals.push(
#             [rat * ld,ld]
#         )
lcldvals = [
    (None, None)
]

In [171]:
# Static and Dynamic ratio
# dynamic_ratio = 1.
# soverd_ratios = [2.]
# soverd = []
# for rat in soverd_ratios:
#     soverd.push(
#         [rat * dynamic_ratio, dynamic_ratio]
#     )
d = None 
ss = [None]
static_vals = []
for v in ss:
    static_vals.append({
        "ClosestToExit" : v
    })

In [172]:
# Aspirations
asp = [None]


In [173]:
# Ratio
ratio = [None]

In [174]:
# FireSpread
f_s = np.around(np.arange(0.1,0.91,0.05),decimals=3)
names = tuple(f"f_s_test_val_{f}.json" for f in f_s)
filedir = "fire_spread"
print(f_s)

[0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9 ]


In [175]:
# Setup values

setups = []
for (n, fs, (lc,ld)) in itertools.product(
    evac_numbers,
    f_s,
    lcldvals
):
    s = {
        "evac_number" : n
    }
    if fs is not None:
        s["fire_spread"] = fs
    if lc is not None:
        s["lc"] = lc
    if ld is not None:
        s["ld"] = ld
    setups.append(s)

In [176]:
# Fire values
fires = []
for (a,r) in itertools.product(
    asp,
    ratio
):
    fire = {
        "movement" : {
            "ClosestDistance" : d 
        },
        "aspiration" : {
            "LogAspiration" : a 
        },
        "ratio" : {
            "Root" : r 
        }
    }
    fires.append(fire)

In [177]:
# Final
jsons = []
for (s, setup, fire) in itertools.product(
    static_vals,
    setups,
    fires,
):
    jsons.append({
        "dim" : dims,
        "param_seed" :param_seed,
        "sim_type" : sim_type,
        "setup" : setup,
        "fire" : fire,
        "escape" : escape,
        "death" : death,
        "static_input" : s
    })
for js in jsons:
    print(js)
    print(validator.is_valid(js))

{'dim': [21, 21], 'param_seed': 10, 'sim_type': 'Total', 'setup': {'evac_number': 44, 'fire_spread': 0.1}, 'fire': {'movement': {'ClosestDistance': None}, 'aspiration': {'LogAspiration': None}, 'ratio': {'Root': None}}, 'escape': 'TimeTracker', 'death': 'AnnounceInput', 'static_input': {'ClosestToExit': None}}
True
{'dim': [21, 21], 'param_seed': 10, 'sim_type': 'Total', 'setup': {'evac_number': 44, 'fire_spread': 0.15}, 'fire': {'movement': {'ClosestDistance': None}, 'aspiration': {'LogAspiration': None}, 'ratio': {'Root': None}}, 'escape': 'TimeTracker', 'death': 'AnnounceInput', 'static_input': {'ClosestToExit': None}}
True
{'dim': [21, 21], 'param_seed': 10, 'sim_type': 'Total', 'setup': {'evac_number': 44, 'fire_spread': 0.2}, 'fire': {'movement': {'ClosestDistance': None}, 'aspiration': {'LogAspiration': None}, 'ratio': {'Root': None}}, 'escape': 'TimeTracker', 'death': 'AnnounceInput', 'static_input': {'ClosestToExit': None}}
True
{'dim': [21, 21], 'param_seed': 10, 'sim_type': 

In [178]:
# Save data
assert len(jsons) == len(names)
dir_to_write_to = f"{path}/{filedir}"
json_obj = None
for (js,n) in zip(jsons, names):
    json_obj = json.dumps(js, indent=4)
    with open(f"{dir_to_write_to}/{n}","w") as out:
        out.write(json_obj)
    json_obj = None

